In [26]:
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
import geodatasets
import matplotlib.pyplot as plt
import plotly.express as px


In [ ]:
# upload collision data
crash_df = pd.read_csv('../Resources/Montgomery_Crash_Data.csv', index_col='Crash Date/Time', parse_dates=True)
crash_df.head()

In [ ]:
# Create dataframe into a geodataframe
montgomery_gdf = gpd.GeoDataFrame(crash_df, geometry=gpd.points_from_xy(crash_df.Longitude, crash_df.Latitude), crs='EPSG:4326').sort_index()
montgomery_gdf.head(3)


In [ ]:
# Get column information
montgomery_gdf.info()

In [ ]:
# upload county Geojson and make it into geodataframe
county_gdf = gpd.read_file('../Resources/Montgomery_County.json')
county_gdf.head()

In [ ]:
# create background map
m = folium.Map(location=[39.1364, -77.1715], tiles="Cartodb dark_matter", zoom_start=11) 

#create geojson with all county boundaries
#add fill color for polygons
for _, r in county_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "none"})
    folium.Popup(r["County"]).add_to(geo_j)
    geo_j.add_to(m)


### Paramters to add to the map ####
# column = None
# value = None
# start_year = '2024'

# column = 'Surface Condition'
# value = 'ICE'
# start_year = '2015'

# column = 'Weather'
# value = 'RAIN'
# start_year = '2015'

column = 'Injury Severity' # column of interest
value = 'FATAL INJURY' # choose the value you want overlay on map
start_year = '2015' #chose start year you want to investigate

if column:
    #filter gdf by year interval and create conditional statement
    filtered_data = montgomery_gdf.loc[start_year:'2024']
    conditional = filtered_data[column] == value
    #create list of point geometries and a heatmap from them
    heat_data = [[point.xy[1][0], point.xy[0][0]] for point in filtered_data.loc[conditional].geometry]
    plugins.HeatMap(heat_data, min_opacity=.2, radius=20,zoom_start=2).add_to(m)

    #create a list dates in string format and attribute them to the points 
    #created on the map
    date_data = [str(date) for date in filtered_data.loc[conditional].index]
    for point, date in zip(heat_data, date_data):
        folium.CircleMarker(point, radius=.2, fill_color="black", color="blue", popup=folium.Popup(date)).add_to(m)
else:
    #filter gdf by year interval
    filtered_data = montgomery_gdf.loc[start_year:'2024']
    #create list of point geometries and a heatmap from them
    heat_data = [[point.xy[1][0], point.xy[0][0]] for point in filtered_data.geometry]
    plugins.HeatMap(heat_data, min_opacity=.2, radius=20,zoom_start=2).add_to(m)

    #create a list dates in string format and attribute them to the points 
    #created on the map
    date_data = [str(date) for date in filtered_data.index]
    for point, date in zip(heat_data, date_data):
        folium.CircleMarker(point, radius=.2, fill_color="black", color="blue", popup=folium.Popup(date)).add_to(m)

#add widget that allows user to turn on and off map layers    
folium.LayerControl().add_to(m)

m

In [ ]:
# make a list for the weeks of the year
weeks = crash_df.index.isocalendar().week.sort_values().unique()

# plot the counts for each week
plt.bar(weeks, crash_df.index.isocalendar().week.value_counts().sort_index())
plt.title('Weekly Crashes')
plt.xlabel('Weeks')
plt.ylabel('Counts')
plt.show()

#plot average chrashes for each week
weekly_crash_count = crash_df.groupby([crash_df.index.isocalendar().year, crash_df.index.isocalendar().week])['Report Number'].count().reset_index()
plt.bar(weeks, weekly_crash_count.groupby('week')['Report Number'].mean().round())
plt.title('Weekly Crashes')
plt.xlabel('Week')
plt.ylabel('Average Crashes')
plt.show()

In [ ]:
# make a list for the days of the week
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']  # crash_df.index.isocalendar().day.sort_values().unique()

# plot the counts for each day
plt.bar(days, crash_df.index.isocalendar().day.value_counts().sort_index())
plt.title('Daily Crashes')
plt.xlabel('Day')
plt.ylabel('Counts')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.show()

#plot average crashes for each day
daily_crash_count = crash_df.groupby([crash_df.index.isocalendar().year, crash_df.index.isocalendar().week, 
                                      crash_df.index.isocalendar().day])['Report Number'].count().reset_index()
plt.bar(days, daily_crash_count.groupby('day')['Report Number'].mean().round())
plt.title('Daily Average Crashes')
plt.xlabel('Day')
plt.ylabel('Average Crashes')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.show()

In [ ]:
# make a list for the weeks of the year
years = crash_df.index.isocalendar().year.sort_values().unique()

# plot the counts for each week
plt.bar(years, montgomery_gdf.index.isocalendar().year.value_counts().sort_index())
plt.grid(linestyle = '-' )
plt.title('Yearly Crashes')
plt.xlabel('Year')
plt.ylabel('Counts')
plt.show()